# Retail Demo Store - Personalization Workshop - Lab 3

In this lab we are going to build on the [prior lab](./Lab-02-(Core)-Prepare-Personalize-and-import-data.ipynb) by creating Amazon Personalize domain recommenders and custom solutions for additional use cases.

## Lab 3 Objectives

In this lab we will accomplish the following steps.

- Create retail domain recommenders for the following use cases:
    - **Recommended For You**: will be used on the homepage (and other later workshops) to provide personalized recommendations for a given user. This recommender will be used for known users or warm anonymous users.
    - **Popular Items by Views**: will be used on the homepage for new/cold anonymous users where no interaction history is available. Once a new/cold user has a few interactions (product views), the homepage will switch to the Recommended For You recommender.
- Create custom solutions and solution versions for the following use cases:
    - **Similar Items**: will be used on the product detail and "live" pages to display similar items to a given item.
    - **Personalized-Ranking**: will be used to rerank featured products and search results.
    - **Item Attribute Affinity**: user segmentation model that will be used in a later lab.
    
This lab should take about 70-90 minutes to complete. However, most of the time will be waiting for model training jobs to complete.

## Setup

Just as in the previous labs, we have to prepare our environment by importing dependencies and creating clients.

### Import dependencies

The following libraries are needed for this lab.

In [ ]:
import boto3
import json
import time
from botocore.exceptions import ClientError

### Create clients

We will need the following AWS service clients in this lab.

In [ ]:
personalize = boto3.client('personalize')

### Load variables saved in prior labs

At the end of Lab 1 we saved some variables that we'll need in this lab. The following cell will load those variables into this lab environment.

In [ ]:
%store -r

## Create Recommenders

With our three datasets imported into our dataset group, we can now turn to creating recommenders and solutions. We'll start with pre-configured recommenders that match some of our core use cases.

Let's start by listing the recipes for the `ECOMMERCE` domain. From these recipes we will be creating the following three recommenders.

- **Recommended For You** - will be used on the home page for the "Inspired by your shopping trends" grid.
- **Popular Items By Views** - will be used on the home page for new/cold users with no interactions so we can bootstrap the user experience with popular products. Once the user is "warm", we will switch to the Recommended For You recommender.

In [ ]:
response = personalize.list_recipes(domain = "ECOMMERCE")
print(json.dumps(response['recipes'], indent=2, default=str))

### Create "Recommended For You" recommender

If you recall from the previous lab where we created the schemas for all three datasets, there were two columns in the items dataset that we wanted to use for filtering (`PROMOTED`) or for generating thematic descriptions (`PRODUCT_NAME`). We'll cover the use cases for these columns in subsequent labs. Since these two columns will not add value to the model as model features, we want to tell Personalize to exclude them from model training. To accomplish this we'll specify a `recommenderConfig` that includes those columns in the `excludedDatasetColumns` declaration.

In [ ]:
# Define training config to use when creating recommenders and solutions.
training_config = {
    "trainingDataConfig": {
        "excludedDatasetColumns": {
            "ITEMS": [ "PRODUCT_NAME", "PROMOTED" ]
        }
    }
}

Next we'll create the recommender, passing the recommender configuration via the `recommenderConfig` argument.

In [ ]:
try:
    response = personalize.create_recommender(
      name = 'retaildemostore-recommended-for-you',
      recipeArn = 'arn:aws:personalize:::recipe/aws-ecomm-recommended-for-you',
      datasetGroupArn = dataset_group_arn,
      recommenderConfig = training_config
    )
    rfy_recommender_arn = response['recommenderArn']
    print(json.dumps(response, indent=2))
except personalize.exceptions.ResourceAlreadyExistsException:
    print('You aready created this recommender, seemingly')
    paginator = personalize.get_paginator('list_recommenders')
    for paginate_result in paginator.paginate(datasetGroupArn = dataset_group_arn):
        for recommender in paginate_result['recommenders']:
            if recommender['name'] == 'retaildemostore-recommended-for-you':
                rfy_recommender_arn = recommender['recommenderArn']
                break
                
print(f'Recommended For You recommender ARN = {rfy_recommender_arn}')

### Create "Popular Items By Views" recommender

When creating the popular items recommender, we use the same recommender config.

In [ ]:
try:
    response = personalize.create_recommender(
      name = 'retaildemostore-popular-items',
      recipeArn = 'arn:aws:personalize:::recipe/aws-ecomm-popular-items-by-views',
      datasetGroupArn = dataset_group_arn,
      recommenderConfig = training_config
    )
    most_viewed_recommender_arn = response['recommenderArn']
    print(json.dumps(response, indent=2))
except personalize.exceptions.ResourceAlreadyExistsException:
    print('You aready created this recommender, seemingly')
    paginator = personalize.get_paginator('list_recommenders')
    for paginate_result in paginator.paginate(datasetGroupArn = dataset_group_arn):
        for recommender in paginate_result['recommenders']:
            if recommender['name'] == 'retaildemostore-popular-items':
                most_viewed_recommender_arn = recommender['recommenderArn']
                break
                
print(f'Most Viewed recommender ARN = {most_viewed_recommender_arn}')

## Create Custom Solutions

The recommenders created above do not cover all of the personalization use cases that we want to implement in the Retail Demo Store. We also want to provide related items recommendations on the product detail page, personalize the order of featured products displayed on the home page, and we want to personalize the search results returned from Open Search in the search widget. To implement these use cases, we will create a custom solution using the [Similar-Items](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-similar-items.html) and [Personalized-Ranking](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-search.html) recipes.

In addition, we also want to use Personalize to create user segments based on user affinity with specific product attributes. We'll explore this is a later lab but let's go ahead and create a custom solution using the [Item-Attribute-Affinity](https://docs.aws.amazon.com/personalize/latest/dg/item-attribute-affinity-recipe.html) recipe.

These custom solutions will use the same datasets that we already implemented so all we need to do is create a solution and solution version for each recipe.

### List Recipes

First, let's list all available recipes that aren't associated with a domain.

In [ ]:
response = personalize.list_recipes()
custom_recipes = []
for recipe in response['recipes']:
    if not recipe.get('domain'):
        custom_recipes.append(recipe)
        
print(json.dumps(custom_recipes, indent=2, default=str))

As you can see above, there are several recipes to choose from. Let's declare the recipes for the two custom solutions we will have to create.

#### Declare Personalize Recipe for Similar Items

In use-cases where we have an item/product and we want to display similar items based on the co-interactions of all users as well as draw upon thematic similarities based on item metadata, we can use the [Similar-Items](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-similar-items.html) recipe to provide related items recommendations.

> The Similar-Items (aws-similar-items) generates recommendations for items that are similar to an item you specify. Use Similar-Items to help customers discover new items in your catalog based on their previous behavior and item metadata. Recommending similar items can increase user engagement, click-through rate, and conversion rate for your application.

> Similar-Items calculates similarity based on interactions data and any item metadata you provide. It takes into account the co-occurrence of the item in user histories in your Interaction dataset, and any item metadata similarities. For example, with Similar-Items Amazon Personalize could recommend items customers frequently bought together with a similar style (Categorical metadata), or movies that different users also watched with a similar description (Unstructured text metadata).

Note that Personalize also has the [SIMS](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html) recipe for the related items use case. However, SIMS only trains on co-interaction data (i.e. the interactions dataset) and does not consider item metadata. Since we may have some items with fewer (or no) interactions, the Similar-Items recipe is a better match for the Retail Demo Store.

In [ ]:
similar_items_recipe_arn = "arn:aws:personalize:::recipe/aws-similar-items"

#### Declare Personalize Recipe for Personalized Ranking

In use-cases where we have a curated list of products, we can use the [Personalized-Ranking](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-search.html) recipe to reorder the products for the current user.

> The Personalized-Ranking recipe generates personalized rankings. A personalized ranking is a list of recommended items that are re-ranked for a specific user.

In [ ]:
ranking_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

#### Declare Personalize Recipe for Item Attribute Affinity

For the user segmentation use case, we will generate groups of users (segments) with an affinity for specific item attributes. We will explore this use case further in an upcoming lab but for now we will create a custom solution and solution version using the [Item-Attribute-Affinity](https://docs.aws.amazon.com/personalize/latest/dg/item-attribute-affinity-recipe.html) recipe.

> The Item-Attribute-Affinity (aws-item-attribute-affinity) recipe is a USER_SEGMENTATION recipe that creates a user segment (group of users) for each item attribute that you specify. Use Item-Attribute-Affinity to learn more about your users and take actions based on their respective user segments.

> For example, you might want to create a marketing campaign for your retail application based on user preferences for shoe types in your catalog. Item-Attribute-Affinity would create a user segment for each shoe type based data in your Interactions and Items datasets. You could use this to promote different shoes to different user segments based on the likelihood that they will take an action (for example, click a shoe or purchase a shoe). Other uses might include promoting different movie genres to different users or identifying prospective job applicant based on job type.

In [ ]:
item_attribute_affinity_recipe_arn = 'arn:aws:personalize:::recipe/aws-item-attribute-affinity'

### Create Custom Solutions and Solution Versions

With our recipes defined, we can now create our solutions and solution versions.

Just like with the recommenders, we will want to tell Personalize to exlude the `PROMOTED` and `PRODUCT_NAME` columns from training. This will be accomplished using the `solutionConfig` argument when creating solutions to pass the training configuration.

#### Create Similar Items Solution

In [ ]:
similar_items_solution_version_arn = None

try:
    create_solution_response = personalize.create_solution(
        name = "retaildemostore-related-items",
        datasetGroupArn = dataset_group_arn,
        recipeArn = similar_items_recipe_arn,
        solutionConfig = training_config
    )

    similar_items_solution_arn = create_solution_response['solutionArn']
    print(json.dumps(create_solution_response, indent=2))
except personalize.exceptions.ResourceAlreadyExistsException:
    print('You aready created this solution, seemingly')
    paginator = personalize.get_paginator('list_solutions')
    for paginate_result in paginator.paginate(datasetGroupArn = dataset_group_arn):
        for solution in paginate_result['solutions']:
            if solution['name'] == 'retaildemostore-related-items':
                similar_items_solution_arn = solution['solutionArn']
                print(f'Similar Items solution ARN = {similar_items_solution_arn}')
                
                response = personalize.list_solution_versions(
                    solutionArn = similar_items_solution_arn,
                    maxResults = 100
                )
                if len(response['solutionVersions']) > 0:
                    similar_items_solution_version_arn = response['solutionVersions'][-1]['solutionVersionArn']
                    print(f'Will use most recent solution version for this solution: {similar_items_solution_version_arn}')
                    
                break


#### Create Similar Items Solution Version

Next we can create a solution version for the solution. This is where the model is trained for this custom solution.

In [ ]:
if not similar_items_solution_version_arn:
    create_solution_version_response = personalize.create_solution_version(
        solutionArn = similar_items_solution_arn
    )

    similar_items_solution_version_arn = create_solution_version_response['solutionVersionArn']
    print(json.dumps(create_solution_version_response, indent=2))
else:
    print(f'Solution version {similar_items_solution_version_arn} already exists; not creating')

#### Create Personalized Ranking Solution

In [ ]:
ranking_solution_version_arn = None

try:
    create_solution_response = personalize.create_solution(
        name = "retaildemostore-personalized-ranking",
        datasetGroupArn = dataset_group_arn,
        recipeArn = ranking_recipe_arn,
        solutionConfig = training_config
    )

    ranking_solution_arn = create_solution_response['solutionArn']
    print(json.dumps(create_solution_response, indent=2))
except personalize.exceptions.ResourceAlreadyExistsException:
    print('You aready created this solution, seemingly')
    paginator = personalize.get_paginator('list_solutions')
    for paginate_result in paginator.paginate(datasetGroupArn = dataset_group_arn):
        for solution in paginate_result['solutions']:
            if solution['name'] == 'retaildemostore-personalized-ranking':
                ranking_solution_arn = solution['solutionArn']
                print(f'Ranking solution ARN = {ranking_solution_arn}')
                
                response = personalize.list_solution_versions(
                    solutionArn = ranking_solution_arn,
                    maxResults = 100
                )
                if len(response['solutionVersions']) > 0:
                    ranking_solution_version_arn = response['solutionVersions'][-1]['solutionVersionArn']
                    print(f'Will use most recent solution version for this solution: {ranking_solution_version_arn}')
                    
                break

#### Create Personalized Ranking Solution Version

Next we can create a solution version for the solution. This is where the model is trained for this custom solution.

In [ ]:
if not ranking_solution_version_arn:
    create_solution_version_response = personalize.create_solution_version(
        solutionArn = ranking_solution_arn
    )

    ranking_solution_version_arn = create_solution_version_response['solutionVersionArn']
    print(json.dumps(create_solution_version_response, indent=2))
else:
    print(f'Solution version {ranking_solution_version_arn} already exists; not creating')

#### Create Item Attribute Affinity Solution

In [ ]:
item_attribute_affinity_solution_version_arn = None

try:
    create_solution_response = personalize.create_solution(
        name = "retaildemostore-item-attribute-affinity",
        datasetGroupArn = dataset_group_arn,
        recipeArn = item_attribute_affinity_recipe_arn,
        solutionConfig = training_config
    )

    item_attribute_affinity_solution_arn = create_solution_response['solutionArn']
    print(json.dumps(create_solution_response, indent=2))
except personalize.exceptions.ResourceAlreadyExistsException:
    print('You aready created this solution, seemingly')
    paginator = personalize.get_paginator('list_solutions')
    for paginate_result in paginator.paginate(datasetGroupArn = dataset_group_arn):
        for solution in paginate_result['solutions']:
            if solution['name'] == 'retaildemostore-item-attribute-affinity':
                item_attribute_affinity_solution_arn = solution['solutionArn']
                print(f'Item Attribute Affinity solution ARN = {item_attribute_affinity_solution_arn}')
                
                response = personalize.list_solution_versions(
                    solutionArn = item_attribute_affinity_solution_arn,
                    maxResults = 100
                )
                if len(response['solutionVersions']) > 0:
                    item_attribute_affinity_solution_version_arn = response['solutionVersions'][-1]['solutionVersionArn']
                    print(f'Will use most recent solution version for this solution: {item_attribute_affinity_solution_version_arn}')
                    
                break

In [ ]:
if not item_attribute_affinity_solution_version_arn:
    create_solution_version_response = personalize.create_solution_version(
        solutionArn = item_attribute_affinity_solution_arn
    )

    item_attribute_affinity_solution_version_arn = create_solution_version_response['solutionVersionArn']
    print(json.dumps(create_solution_version_response, indent=2))
else:
    print(f'Solution version {item_attribute_affinity_solution_version_arn} already exists; not creating')

## Wait for Recommenders and Solution Versions to Complete

It can take 40-60 minutes for all recommenders and solution versions to be created. During this process a model is being trained and tested with the data contained within your datasets. The duration of training jobs can increase based on the size of the dataset, training parameters and a selected recipe. We submitted requests for all three recommenders and two custom solutions and versions at once so they are trained in parallel. In the cells below we will wait for all recommenders and solution versions to finish.

While you are waiting for this process to complete you can learn more about [recommenders](https://docs.aws.amazon.com/personalize/latest/dg/creating-recommenders.html) and [custom solutions](https://docs.aws.amazon.com/personalize/latest/dg/training-deploying-solutions.html).

The following cell waits for all recommenders to become active.

In [ ]:
%%time

recommender_arns = [ rfy_recommender_arn, most_viewed_recommender_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for recommender_arn in reversed(recommender_arns):
        response = personalize.describe_recommender(
            recommenderArn = recommender_arn
        )
        status = response["recommender"]["status"]

        if status == "ACTIVE":
            print(f'Recommender {recommender_arn} successfully completed')
            recommender_arns.remove(recommender_arn)
        elif status == "CREATE FAILED":
            print(f'Recommender {recommender_arn} failed')
            if response["recommender"].get('failureReason'):
                print('   Reason: ' + response["recommender"]['failureReason'])
            recommender_arns.remove(recommender_arn)

    if len(recommender_arns) > 0:
        print('At least one recommender is still in progress')
        time.sleep(60)
    else:
        print("All recommenders have completed")
        break

#### Wait for custom solution versions to become active

The following cell waits for the solution versions for the similar items, personalized ranking, and item attribute affinity use cases to become active. It's likely that they're already active (or close to being active) since they were being created in parallel with the recommenders. Nevertheless, we'll make sure they are active too before proceeding.

In [ ]:
%%time

soln_ver_arns = [ 
    similar_items_solution_version_arn, 
    ranking_solution_version_arn, 
    item_attribute_affinity_solution_version_arn 
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for soln_ver_arn in reversed(soln_ver_arns):
        soln_ver_response = personalize.describe_solution_version(
            solutionVersionArn = soln_ver_arn
        )
        status = soln_ver_response["solutionVersion"]["status"]

        if status == "ACTIVE":
            print(f'Solution version {soln_ver_arn} successfully completed')
            soln_ver_arns.remove(soln_ver_arn)
        elif status == "CREATE FAILED":
            print(f'Solution version {soln_ver_arn} failed')
            if soln_ver_response["solutionVersion"].get('failureReason'):
                print('   Reason: ' + soln_ver_response["solutionVersion"]['failureReason'])
            soln_ver_arns.remove(soln_ver_arn)

    if len(soln_ver_arns) > 0:
        print('At least one solution version is still in progress')
        time.sleep(60)
    else:
        print("All solution versions have completed")
        break

### Evaluate Offline Metrics for Recommenders and Custom Solution Versions

Amazon Personalize provides [offline metrics](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html#working-with-training-metrics-metrics) for recommenders and custom solutions that allow you to evaluate the accuracy of the model before you deploy the model in your application. Metrics can also be used to view the effects of modifying a custom solution's hyperparameters or to compare the metrics between solutions that use the same training data but created with different recipes.

Let's retrieve the metrics for the recommenders and custom solution versions we just created.

#### Recommended For You Recommender Metrics

In [ ]:
response = personalize.describe_recommender(
    recommenderArn = rfy_recommender_arn
)

print(json.dumps(response['recommender']['modelMetrics'], indent=2, default=str))

#### Popular Viewed Recommender Metrics

In [ ]:
response = personalize.describe_recommender(
    recommenderArn = most_viewed_recommender_arn
)

print(json.dumps(response['recommender']['modelMetrics'], indent=2, default=str))

#### Similar Items Metrics

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = similar_items_solution_version_arn
)

print(json.dumps(get_solution_metrics_response['metrics'], indent=2))

#### Personalized Ranking Metrics

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = ranking_solution_version_arn
)

print(json.dumps(get_solution_metrics_response['metrics'], indent=2))

#### Item Attribute Affinity Metrics

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = item_attribute_affinity_solution_version_arn
)

print(json.dumps(get_solution_metrics_response['metrics'], indent=2))

You can learn more about interpreting offline metrics in the Personalize [documentation](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) and in the blog post on [A/B testing](https://aws.amazon.com/blogs/machine-learning/using-a-b-testing-to-measure-the-efficacy-of-recommendations-generated-by-amazon-personalize/) with Personalize.

## Create campaigns for Similar Items and Personalized Ranking solutions

Once we're satisfied with our solution versions, we need to create campaigns for the custom solution versions created for the similar items and personalized ranking recipes. This is required so we have an real-time API endpoints that can be called from the Recommendations microservice to return related items for the product detail page and rerank products for the featured products and search widgets. When creating a campaign you can specify the minimum transactions per second (`minProvisionedTPS`) that you expect to make against the service for this campaign. Personalize will automatically scale resources for the inference endpoint up and down for the campaign to match demand but will never scale below `minProvisionedTPS`.

Let's create campaigns for the similar items and personalized ranking custom solution versions with each set at `minProvisionedTPS` of 1 (which is also the default if not specified).

Note: For the item attribute affinity custom solution, we don't need a campaign since it only supports generating user segments using batch segmentation jobs. We'll explore this in a later lab.

#### Create Similar Items campaign

In [ ]:
try:
    create_campaign_response = personalize.create_campaign(
        name = "retaildemostore-related-items",
        solutionVersionArn = similar_items_solution_version_arn,
        minProvisionedTPS = 1
    )

    similar_items_campaign_arn = create_campaign_response['campaignArn']
    print(json.dumps(create_campaign_response, indent=2))
except personalize.exceptions.ResourceAlreadyExistsException:
    print('You aready created this campaign, seemingly. Will update campaign instead.')
    paginator = personalize.get_paginator('list_campaigns')
    for paginate_result in paginator.paginate(solutionArn = similar_items_solution_arn):
        for campaign in paginate_result['campaigns']:
            if campaign['name'] == 'retaildemostore-related-items':
                similar_items_campaign_arn = campaign['campaignArn']
                print(f'Found existing campaign for solution: {similar_items_campaign_arn}')
                
                response = personalize.describe_campaign(campaignArn = similar_items_campaign_arn)
                if response['campaign']['solutionVersionArn'] == similar_items_solution_version_arn:
                    print('Campaign is already using the latest solution version')
                else:
                    print('Updating campaign with the latest solution version')
                    response = personalize.update_campaign(
                        campaignArn = similar_items_campaign_arn,
                        solutionVersionArn = similar_items_solution_version_arn,
                        minProvisionedTPS = 1
                    )
                    print(json.dumps(response, indent=2))
                break

#### Create Personalized Ranking campaign

In [ ]:
try:
    create_campaign_response = personalize.create_campaign(
        name = "retaildemostore-personalized-ranking",
        solutionVersionArn = ranking_solution_version_arn,
        minProvisionedTPS = 1
    )

    ranking_campaign_arn = create_campaign_response['campaignArn']
    print(json.dumps(create_campaign_response, indent=2))
except personalize.exceptions.ResourceAlreadyExistsException:
    print('You aready created this campaign, seemingly. Will update campaign instead.')
    paginator = personalize.get_paginator('list_campaigns')
    for paginate_result in paginator.paginate(solutionArn = ranking_solution_arn):
        for campaign in paginate_result['campaigns']:
            if campaign['name'] == 'retaildemostore-personalized-ranking':
                ranking_campaign_arn = campaign['campaignArn']
                print(f'Found existing campaign for solution: {ranking_campaign_arn}')
                
                response = personalize.describe_campaign(campaignArn = ranking_campaign_arn)
                if response['campaign']['solutionVersionArn'] == ranking_solution_version_arn:
                    print('Campaign is already using the latest solution version')
                else:
                    print('Updating campaign with the latest solution version')
                    response = personalize.update_campaign(
                        campaignArn = ranking_campaign_arn,
                        solutionVersionArn = ranking_solution_version_arn,
                        minProvisionedTPS = 1
                    )
                    print(json.dumps(response, indent=2))
                break

#### Wait for campaigns to Have ACTIVE Status

It can take 15-20 minutes for a campaign to be fully created. 

While you are waiting for this to complete you can learn more about campaigns here: https://docs.aws.amazon.com/personalize/latest/dg/campaigns.html

In [ ]:
%%time

campaign_arns = [ similar_items_campaign_arn, ranking_campaign_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for campaign_arn in reversed(campaign_arns):
        campaign_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = campaign_response["campaign"]["status"]
        if status == 'ACTIVE' and campaign_response.get('latestCampaignUpdate'):
            status = campaign_response['latestCampaignUpdate']['status']

        if status == "ACTIVE":
            print(f'Campaign {campaign_arn} successfully completed')
            campaign_arns.remove(campaign_arn)
        elif status == "CREATE FAILED":
            print(f'Campaign {campaign_arn} failed')
            if campaign_response["campaign"].get('failureReason'):
                print('   Reason: ' + campaign_response["campaign"]['failureReason'])
            campaign_arns.remove(campaign_arn)

    if len(campaign_arns) > 0:
        print('At least one campaign is still in progress')
        time.sleep(60)
    else:
        print("All campaigns have completed")
        break

## Lab 3 Summary - What have we accomplished?

In this lab we created retail domain recommenders for different personalization use cases. We also created custom solutions for similar items and personalized ranking as well as item attribute affinity (user segmentation) use cases.

In the next lab we will examine the recommendations provided by the recommenders and the custom campaigns.

### Store variables needed in the next lab

We will pass some variables initialized in this lab by storing them in the notebook environment.

In [ ]:
# Store recommender ARNs
%store rfy_recommender_arn
%store most_viewed_recommender_arn

# Store solution version ARNs
%store similar_items_solution_version_arn
%store ranking_solution_version_arn
%store item_attribute_affinity_solution_version_arn

# Store campaign ARNs
%store similar_items_campaign_arn
%store ranking_campaign_arn

Continue to [Lab 4](./Lab-04-(Core)-Evaluate-recommendations.ipynb).